In [1]:
!pip install sagemaker

In [2]:
import sagemaker
import boto3

# Initialize SageMaker session
sess = sagemaker.Session()

# Set up the SageMaker session bucket
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

# Manage the SageMaker execution role
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Set up a SageMaker session with the default bucket
session = sagemaker.Session(default_bucket=sagemaker_session_bucket)

# Print the SageMaker role ARN and session region
print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::445567117176:role/service-role/AmazonSageMaker-ExecutionRole-20241105T020132
sagemaker session region: us-east-1


In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

# Define the Hugging Face model and task
hub = {
    'HF_MODEL_ID': 'distilbert-base-uncased-distilled-squad',
    'HF_TASK': 'question-answering'
}

# Initialize the Hugging Face model
huggingface_model = HuggingFaceModel(
    env=hub,
    role=role,
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39",
)

# Deploy the model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.t3.medium"
)

# Example request data
data = {
    "inputs": {
        "question": "What is used for inference?",
        "context": "My Name is Philipp and I live in Nuremberg. This model is used with SageMaker for inference."
    }
}

# Send a prediction request
prediction = predictor.predict(data)
print(prediction)
